In [1]:
import traceback
import sys
 
from gribapi import *

ECMWF python grib-api example
https://software.ecmwf.int/wiki/display/GRIB/count_messages.py

In [9]:
INPUT='Z__C_RJTD_20160227120000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin'
VERBOSE=1 # verbose error reporting
 
def example():
    f = open(INPUT)
 
    mcount = grib_count_in_file(f)
    gid_list = [grib_new_from_file(f) for i in range(mcount)]
 
    f.close()
 
    keys = [
        'Ni',
        'Nj',
        'latitudeOfFirstGridPointInDegrees',
        'longitudeOfFirstGridPointInDegrees',
        'latitudeOfLastGridPointInDegrees',
        'longitudeOfLastGridPointInDegrees',
        'jDirectionIncrementInDegrees',
        'iDirectionIncrementInDegrees',
        ]
 
    for i in range(mcount):
        gid = gid_list[i]
 
        print "processing message number",i+1
        print gid
 
        for key in keys:
            print '%s=%g' % (key,grib_get(gid,key))
 
        print 'There are %d, average is %g, min is %g, max is %g' % (
                  grib_get_size(gid,'values'),
                  grib_get(gid,'average'),
                  grib_get(gid,'min'),
                  grib_get(gid,'max')
               )
 
        print '-'*100
 
        grib_release(gid)
 
 
def main():
    try:
        example()
    except GribInternalError,err:
        if VERBOSE:
            traceback.print_exc(file=sys.stderr)
        else:
            print >>sys.stderr,err.msg
 
        return 1

main()

processing message number 1
4
Ni=2560
Nj=3360
latitudeOfFirstGridPointInDegrees=47.9958
longitudeOfFirstGridPointInDegrees=118.006
latitudeOfLastGridPointInDegrees=20.0042
longitudeOfLastGridPointInDegrees=149.994
jDirectionIncrementInDegrees=0.008333
iDirectionIncrementInDegrees=0.0125


Traceback (most recent call last):
  File "<ipython-input-9-b93ee17acff3>", line 46, in main
    example()
  File "<ipython-input-9-b93ee17acff3>", line 33, in example
    grib_get_size(gid,'values'),
  File "/usr/lib/pymodules/python2.7/gribapi.py", line 70, in modified
    return _func_(**kw)
  File "/usr/lib/pymodules/python2.7/gribapi.py", line 385, in grib_get_size
    GRIB_CHECK(err)
  File "/usr/lib/pymodules/python2.7/gribapi.py", line 70, in modified
    return _func_(**kw)
  File "/usr/lib/pymodules/python2.7/gribapi.py", line 143, in GRIB_CHECK
    raise GribInternalError(errid)
GribInternalError: Key/value not found


1

In [8]:
def example():
    f = open(INPUT)
 
    while 1:
        gid = grib_new_from_file(f)
        if gid is None: break
 
        iterid = grib_keys_iterator_new(gid,'ls')
 
        # Different types of keys can be skipped
        # grib_skip_computed(iterid)
        # grib_skip_coded(iterid)
        # grib_skip_edition_specific(iterid)
        # grib_skip_duplicates(iterid)
        # grib_skip_read_only(iterid)
        # grib_skip_function(iterid)
 
        while grib_keys_iterator_next(iterid):
            keyname = grib_keys_iterator_get_name(iterid)
            keyval = grib_get_string(iterid,keyname)
            print "%s = %s" % (keyname,keyval)
 
        grib_keys_iterator_delete(iterid)
        grib_release(gid)
 
    f.close()
 
def main():
    try:
        example()
    except GribInternalError,err:
        if VERBOSE:
            traceback.print_exc(file=sys.stderr)
        else:
            print >>sys.stderr,err.msg
 
        return 1
 
main()

edition = 2
centre = rjtd
date = 20160227
dataType = an
gridType = regular_ll


gribapiでもradarは不可